In [2]:
# Carefully modify the below two string variables. Ensure there are no typos.

student_id = "11093712" # set this to your student ID

student_mail = "matthew.crean@student.manchester.ac.uk" # your email address

# Coursework 3

This coursework test contains several Jupyter Notebook cells with the comment `# TODO`. This is where you type the code for your solutions. Do not alter any of the other cells. 

It is good practice to include markdown cells explaining your work, but in this test they won't be marked. 

Here are some tips:

* **Do not alter the names of the predefined variables and functions,** such as `itls`, `CoD`, etc. The (return) values of these variables and functions will inform the marking. Renaming them and failure to follow the problem description will result in loss of marks.

* **Ensure that functions *return* values, not merely print them.** Each function should have at least one occurance of the `return` keyword, followed by a variable of the type required by the question.  

* **Do not hard-code any solution variables.** All problems must be solved by computer code using the data in the provided CSV file. For example, do *not* simply define a variable `cod = 1234` with a fixed value. Your Jupyter Notebook should produce results with modified data that has the same format but different numerical (or NaN) values.

* **Avoid inefficient computations.** Ensure that each cell can be run in about 20 seconds on a modern laptop, except some other timeout is stated. Long-running cells will be timed out which will result in loss of marks.

* **Submit this test as a single .ipynb file using Blackboard.** You can simply keep the name `test3-2025.ipynb`. There is a basic testing code at the end that verifies some parts of the coursework.

   <span style="color:blue; font-weight:bold">Strict deadline: Monday, 5th of May 2025, at 1pm. There are no automatic extensions.</span>

### Note on independent work

You need to complete all coursework tests independently on your own, but you are allowed to use online resources and all course notes and exercise solutions. The course notes from chapters 1 to 5 contain all that is required to solve the below problems. You are not allowed to ask other humans for help. In particular, you are not allowed to send, give, or receive code or markdown content to/from classmates and others.

The University Guidelines for Academic Malpractice apply: http://documents.manchester.ac.uk/display.aspx?DocID=2870

**Important: Even if you are the originator of the work** (and not the one who copied), the University Guidelines require that you will be equally responsible for this case of academic malpractice and may lose all coursework marks (or even be assigned 0 marks for the course).

# Start of test

In [3]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=DeprecationWarning)

import numpy as np
import pandas as pd

## Problem 1a

We have seen that LASSO tends to produce sparse weight vectors that approximately minimize $\| \mathbf X \mathbf w - \mathbf y\|_2^2$. 

An alternative approach to solving the sparse regression problem is *iterative thresholding*. In this approach, we first compute a minimum-norm least squares solution $\mathbf w^{(0)}$ that minimizes $\| \mathbf X \mathbf w - \mathbf y\|_2^2$ without regularisation. Then, we identify the components of $\mathbf w^{(0)}$ that are below a certain threshold $\tau\geq 0$ in modulus, and remove the corresponding features from $\mathbf{X}$ to obtain  $\mathbf{X}^{(1)}$. We then minimize  $\| \mathbf X^{(1)} \mathbf w - \mathbf y\|_2^2$ to get $\mathbf{w}^{(1)}$, and so on, until all components in the weight vector $\mathbf{w}^{(k)}$ are $\geq \tau$ in modulus. 

Implement this iterative thresholding algorithm in a function `itls(X, y, tau)` using only plain Python and NumPy. Here, `X` is an $n\times d$ NumPy array, `y` is a NumPy vector of length $n$, and `tau` is the threshold (a floating point number $\geq 0$). The function should return the (sparse) $d$-dimensional weight vector $\mathbf w$.

In [4]:
def itls(X,y,tau):
    d = X.shape[1]
    w = np.linalg.lstsq(X,y)[0]
    threshold_old = np.abs(w) >= tau #this has length d and we want this to be the final one we apply to get w (update at each step)

    while True:
        w1 = np.linalg.lstsq(X[:, threshold_old],y)[0]

        w0 = np.zeros(d)

        w0[threshold_old] = w1 #UNSURE, should be changing all true values to the corresponding w1 values and keeping the rest at 0

        threshold_new = np.abs(w0) >= tau

        if np.array_equal(threshold_new, threshold_old):
            return w0

        threshold_old = threshold_new

## Problem 1b

Write a function `itls1(X, y, tau)` like in the previous problem but with the following modification: at each iteration, instead of zeroing *all* components of the weight vector below the threshold, only the single component which is below the threshold *and smallest in modulus* is set to zero. (If there are several components of the same minimal modulus, only the one with the smallest index should be zeroed.) 

As above, `X` is an $n\times d$ NumPy array, `y` is a NumPy vector of length $n$, and `tau` is the threshold (a floating point number $\geq 0$). The function should return the (sparse) $d$-dimensional weight vector $\mathbf w$.

In [5]:
def itls1(X,y,tau):
    d = X.shape[1]
    remaining_columns = np.ones(d, dtype=bool)
    while True:
        XC = X[:, remaining_columns]
        w_new = np.linalg.lstsq(XC,y)[0]

        w_full = np.zeros(d)

        w_full[remaining_columns] = w_new

        remove = np.where( remaining_columns & (np.abs(w_full) < tau) )[0]

        if len(remove) == 0:
            return w_full
        
        min_value = np.abs(w_full)[remove].min()

        remaining_columns[ np.where( np.abs(w_full) == min_value )[0][0] ] = False

## Problem 1c

Produce a pandas dataframe `CoD` with two columns `itls` and `itls1` and an index with values $5,10,15,\ldots,50$ corresponding to the $\tau$ threshold. The entries of `CoD` correspond to the coefficient of determination (CoD) of the predictions produced with the weights from `itls` and `itls1`, respectively, on the dataset loaded below.

In [6]:
from sklearn import datasets
diabetes = datasets.load_diabetes(as_frame=True)["frame"]
X = diabetes.drop("target", axis=1).values
X = (X - np.mean(X, axis=0)) / np.std(X, axis=0) # z-normalize
y = diabetes["target"].values

In [7]:
sample_mean = y.sum() / len(y)
y_bar = sample_mean*np.ones(len(y))
denominator = (np.square(y - y_bar)).sum()

itls_list = []
itls1_list = []
for i in range(10):
    w = itls(X,y,5*(i+1))
    w1 = itls1(X,y,5*(i+1))
    y_hat = np.matmul(X,w)
    y_hat1 = np.matmul(X,w1)
    itls_list.append(1 - (np.square(y - y_hat).sum())/denominator)
    itls1_list.append(1 - (np.square(y - y_hat1).sum())/denominator)

CoD = pd.DataFrame({"itls" : itls_list, "itls1" : itls1_list}, index = [5,10,15,20,25,30,35,40,45,50])

## Problem 2

The code below loads a corpus of 11,314 newsgroups posts on 20 topics and *vectorizes* them by counting the 500 most frequently occuring words. This results in a NumPy feature array `X_train` of size $11,314 \times 500$. Associated with this is an array `y_train` with entries in $\{ 0,1,\ldots, 19 \}$ labelling the topic of each of the posts. 

Train an sklearn pipeline `newspipe` that provides a `newspipe.predict(X)` method that, when given a $\ell \times 500$ NumPy array of features `X`, returns an $\ell \times 20$ NumPy array of predicitions. Each row of the returned array should be a probability vector, indicating the probability of a newsgroup post to belong to each of the 20 categories.

**Notes:** 
* The first call of `fetch_20newsgroups` might take several minutes to run as the corpus needs to be downloaded from the web. This happens only once. 

* If you're curious about the individual posts, they are stored in the list `corpus`. The 500 most frequent words (of at least 3 characters length) are stored in the array `words`.

* Training and prediction times should be below 40 seconds on a standard laptop, respectively, to avoid timeouts.

In [8]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer

corpus, y_train = fetch_20newsgroups(return_X_y=True)
vectorizer = CountVectorizer(token_pattern=r"\b[a-zA-Z]{3,}\b", max_features=500)
X_train = vectorizer.fit_transform(corpus)
words = vectorizer.get_feature_names_out()
X_train = np.array(X_train.todense())

In [9]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import HistGradientBoostingClassifier

newspipe = Pipeline([
    ('classifier', HistGradientBoostingClassifier(
        max_iter=10,
        max_depth=5,
        random_state=3383,
        max_leaf_nodes=5
    ))
])

newspipe.fit(X_train, y_train)

newspipe.predict = newspipe.predict_proba

newspipe.predict(X_train)

array([[0.01708968, 0.03753402, 0.0190741 , ..., 0.01728031, 0.01796311,
        0.01464076],
       [0.0324305 , 0.06113565, 0.03619627, ..., 0.03279226, 0.03668459,
        0.02778327],
       [0.01550519, 0.0470155 , 0.01730562, ..., 0.01567815, 0.03781937,
        0.01827496],
       ...,
       [0.02439097, 0.0334673 , 0.02879699, ..., 0.02608883, 0.02711968,
        0.02210379],
       [0.01495633, 0.32465525, 0.01069138, ..., 0.00968593, 0.01163011,
        0.00820641],
       [0.04654874, 0.05965459, 0.04647492, ..., 0.03615775, 0.03758646,
        0.03063469]])

## Problem 3

Consider the below loaded dataframes `eur` and `gbp` which store historical GBP-EUR and GBP-USD exchange rates. For each day, the dataframes list the exchange rate at trade closing time, the daily highest and lowest rates, and the rate at trade opening time. The dataframe `rates` combines the data from `eur` and `gbp` into a single dataframe. The task is to predict a day's exchange rates *using only information from previous days*.

Write two functions `train_models` and `predict_rates` as follows.

The function `train_models(rates_t)` takes as input a training dataframe `rates_t` of historical exchange rates. The format `rates_t` is the same as `rates`, but the date range might be different. For example, your `train_models` function should work if only historical data from 2017 until 2023 is provided. But you can always assume that a continuous subset of at least 500 rows is provided. The function returns a dictionary `models` of trained models. This might be a single model or multiple models, completely up to you. The only requirement is that all model training happens in that function and should complete in at most 40 seconds on a standard laptop (for the full dataset of all historical values).

The function `predict_rates(models, rates_h)` takes as inputs the dictionary of trained models and a dataframe `rates_h` of historical data which you can assume to be a continuous subset of at least 50 rows from `rates`. The function returns a Python list of eight floating point values, corresponding to next trading day predictions of `[EUR_Close, EUR_High, EUR_Low, EUR_Open, USD_Close, USD_High, USD_Low, USD_Open]`. It should not take longer than 10 seconds to return a prediction.

**Notes:** 
* Training time on the full dataset should be below 40 seconds on a standard laptop, and prediction time should be at most 10 seconds. 

* Even though `train_models` will be provided with at least 500 trading days of data, you do not need to use all of them for the training. Similarly for the prediction.

In [10]:
# do not change code in this cell
eur = pd.read_csv('_datasets/gbp-eur2.csv', index_col='Date')
usd = pd.read_csv('_datasets/gbp-usd2.csv', index_col='Date')
rates = pd.concat([eur, usd], axis=1, keys=['EUR', 'USD'])
rates.columns = ['_'.join(col).strip() for col in rates.columns.values] 
rates.dropna(inplace=True)
rates.head()

,EUR_Close,EUR_High,EUR_Low,EUR_Open,USD_Close,USD_High,USD_Low,USD_Open
Date,,,,,,,,
2003-12-01,1.4366,1.4405,1.4323,1.4327,1.718597,1.727205,1.718390,1.723811
2003-12-02,1.4308,1.4399,1.4297,1.4368,1.730313,1.732112,1.717298,1.719010
2003-12-03,1.4251,1.4322,1.4249,1.4306,1.728101,1.731812,1.725209,1.730493
2003-12-04,1.4242,1.4300,1.4190,1.4253,1.720697,1.728997,1.718302,1.727414
2003-12-05,1.4213,1.4265,1.4194,1.4243,1.733102,1.733102,1.719809,1.720608


In [11]:
from sklearn.linear_model import Ridge

def train_models(rates_t):
    df = rates_t.copy()

    X = df.values[:-1]
    y = df.values[1:]

    models = {}

    c = df.columns

    for i in range(len(c)):
        model = Ridge(alpha=0.1)
        model.fit(X, y[:,i])
        models[c[i]] = model
        
    return models

def predict_rates(models, rates_h):
    df = rates_h.copy()

    final_date = df.values[-1].reshape(1,-1)

    predict = []

    for col in df.columns:
        next_predict = models[col].predict(final_date)[0]
        predict.append(next_predict)

    return predict

**Example:** This is how your functions should be usable. 
```{python}
# training data
rates_t = rates.loc['2017-01-01':'2023-12-31']
models = train_models(rates_t)

# get some historic data
rates_h = rates.loc['2024-01-01':'2024-08-09']

# predict next row
lst = predict_rates(models, rates_h)

# lst should contain 8 predictions for 2024-08-12, 
# the next trading day following 2024-08-09
```

# End of test

You can use the below tests to get an indication if part of your work returns the right data types.

In [12]:
try: 
    import re
    assert re.match(r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$', student_mail) and not 'firstname' in student_mail
    print("OKAY - student_mail appears to be valid")
except:
    print("WARN - student_mail could not be verified")

import numpy as np
X = np.array([[1,2,3],[2,-3,4],[1,2,3],[2,2,2]])
Y = np.array([[1,2],[1,2],[3,4],[1,2]])
x = np.array([1,2,3])

try: 
    assert callable(itls)
    print("OKAY - itls should be a function")
except:
    print("FAIL - itls should be a function")

X = np.array([[1,2,3],[2,-3,4],[1,2,3],[2,2,2]])
y = np.array([1,2,3,4])

try:
    w = itls(X, y, 0.2)
    assert type(w) == np.ndarray
    print("OKAY - itls returns a NumPy array")
except:
    print("FAIL - itls does not return a NumPy array")

try: 
    assert callable(itls1)
    print("OKAY - itls1 should be a function")
except:
    print("FAIL - itls1 should be a function")

try: 
    assert type(CoD) == pd.DataFrame
    print("OKAY - CoD should be a pandas dataframe")
except:
    print("FAIL - CoD should be a pandas dataframe")

try: 
    assert callable(newspipe.predict)
    print("OKAY - newspipe provides predict method")
except:
    print("FAIL - newspipe does not provide predict method")

try: 
    assert callable(train_models)
    print("OKAY - train_models should be a function")
except:
    print("FAIL - train_models should be a function")

try: 
    assert callable(predict_rates)
    print("OKAY - predict_rates should be a function")
except:
    print("FAIL - predict_rates should be a function")

OKAY - student_mail appears to be valid
OKAY - itls should be a function
OKAY - itls returns a NumPy array
OKAY - itls1 should be a function
OKAY - CoD should be a pandas dataframe
OKAY - newspipe provides predict method
OKAY - train_models should be a function
OKAY - predict_rates should be a function
